# Predicting national health from subjectiv measure on a global scale



In [1]:

import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from sklearn import linear_model
from scipy.stats import pearsonr
pd.options.display.max_columns=None
pd.options.display.max_rows=150

In [2]:
queastions=pd.read_csv("data/question_table.csv")
data_individual=pd.read_csv("data/data_individual_lvl.csv")
data_country=pd.read_csv("data/data_country_lvl.csv")

# presentation 
How could we try to estimate the weelbeing of different nationas? well a good starting point is to ask! i invite you to follow along on an journey of exploration, narreted by me :)


### World happiness report

the methodologcly for world happiness report(whr).

"Life evaluations from the Gallup World Poll provide the basis for the annual happiness rankings. They are based on answers to the main life evaluation question. The Cantril Ladder asks respondents to think of a ladder, with the best possible life for them being a 10 and the worst possible life being a 0. They are then asked to rate their own current lives on that 0 to 10 scale. The rankings are from nationally representative samples over three years."

From world happiness report website.

### World Values Survey



this is the methology of world Values Survey(wvs).

"The World Values Survey (www.worldvaluessurvey.org) is a global network of social scientists studying changing values and their impact on social and political life, led by an international team of scholars, with the WVS association and secretariat headquartered in Stockholm, Sweden. 

The survey, which started in 1981, seeks to use the most rigorous, high-quality research designs in each country. The WVS consists of nationally representative surveys conducted in almost 100 countries which contain almost 90 percent of the world’s population, using a common questionnaire. The WVS is the largest non-commercial, cross-national, time series investigation of human beliefs and values ever executed, currently including interviews with almost 400,000 respondents. Moreover the WVS is the only academic study covering the full range of global variations, from very poor to very rich countries, in all of the world’s major cultural zones.

The WVS seeks to help scientists and policy makers understand changes in the beliefs, values and motivations of people throughout the world"


From world values survey website.

## rank comperison 


### creating own rank
in order to establish a rank i first need to consider what metric i think will be of intrest. after looking through WVS almost 1000 different variables and 450 page variable explonation i selected 3 that i find good. it was 
1.  Feeling of happiness, 'A009'
2.  Satisfaction with your life, 'A170'
3.  State of health (subjective), 'A008'

oskars_rank_with_health: is all three question, even perceived health.

oskars_rank: is just the two first question, excluding percevied health, as this could take into account physical health and nut just mental health


In order to compare against eachother and other variables i have both inverted and standardize neacessary columns values. 

    for columns in df.columns: 
        if columns in (list_of_columns_inverting):
            max=df[columns].max()
            test2=(df[columns]*-1)+max+1 #invert the values
            df[columns]=test2+(test2-1)*((10-max)/(max-1)) #normalize values to 1-10 from x


for futher details see creating_table_one_process where all relevant df and list are listed




In [3]:
data_country['oskars_rank_with_health']=data_country[['A009','A008','A170']].sum(axis=1).rank(ascending=False)
data_country['oskars_score_with_health']=data_country[['A009','A008','A170']].sum(axis=1)/3
data_country['oskars_rank']=data_country[['A009','A170']].sum(axis=1).rank(ascending=False)
data_country['oskars_score']=data_country[['A009','A170']].sum(axis=1)/2
data_country['life_satisfaction_rank']=data_country['A170'].rank(ascending=False)


In [4]:
data_rank=data_country[['oskars_rank_with_health','oskars_rank','country_name','world_happiness_report_rank','life_satisfaction_rank']].sort_values(by='world_happiness_report_rank')
data_rank['diff']=data_rank['oskars_rank_with_health']-data_rank['world_happiness_report_rank']

In [ ]:
data_rank

*note that some of the outliers can be because of error in data and when the different data was collected. an example of this is israel, where the data doesn't exist for one of the variables('A009'). an fix for this and to in crease the robustness of my analysis would be to have avg of the three different columns instead of having a static sum/3.

### plotting the data  as scatter plot

Lets try plotting it as a scatter to try to see if there is a correlation between them :)

In [ ]:
data_rank.plot(x='oskars_rank_with_health',y='world_happiness_report_rank',kind='scatter')

okey that looks bad, like zero correlation :/ 

so instead of doing rank lets try the raw score to see if there is a correlation:)

In [ ]:
data_country.plot(x='oskars_score_with_health',y='Happiness score',kind='scatter')

okey that looks a lot better. forewards lets use the score instead of the rank

## correlation analysis

Lets introduce the metrics of health that we have. i have choosen to focus on 4 different.
1. Dailys_per_100000: It's the amount of year lost to disablility per 100000 from non communicable disieases in a country. it's a rough estimate of the health of the country.
2. deppresion_percent: it's the precentage of the popluation with clinical deppresion, is a measure of "unhappiness"
3. crude_suicide_per_100,000: it's the suicide rate per year per 100000 people. it's also a measure of "extreme stress" and "unhappiness"
4. Life expectancy: it's the amount of year a person is expected to live, a measure of health.

in order to do analysis i slim down the dataset and do correlation on that 

In [8]:
slim_down_list=['Happiness score',
    'oskars_score_with_health',
    'oskars_score',
    'A009',
    'A008',
    'A170',
    'A173',
    'C006',
    'H001',
    'Unemployment rate',
    'Dailys_per_100000',
    'deppresion_percent',
    'crude_suicide_per_100,000',
    'Life expectancy']

In [9]:
data_slim=data_country[slim_down_list]

In [ ]:
print(queastions[['question','description']].loc[queastions['question'].isin(slim_down_list)])

 okey let's do some analysis. it's a reagular correlation matrix and i have added stars* to mark a significant value. the values are 95%*(one star), 99%**(two stars) and 99.9%***(three stars) sure of an actual correlation. -->

In [ ]:
rho = data_slim.corr()
pval = data_slim.corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*rho.shape)
p = pval.map(lambda x: ''.join(['*' for t in [.05, .01, .001] if x<=t]))
rho.round(2).astype(str) + p

Intressting, there are some very strong correlations here. fore example is happiness score a stronbg correlation with both Life expectancy and dalys. but they also correlate psoitive with suicide, so the mopre happy a country is the more suicides happen? that doesnt sound right. if we look at my score then it's actually signifiklant negative correlation with suicide, that's a good sign. let's follow to the next and final step, linear regression. :)

## Doing Linear Regression

### def stepwise linearRegression alghoritm

This part is where i define the the algorithm for selecting variables for the regression analysis, with a stepwise approach. what this does is it loops over a model with all variables and remove the one with the highest pvalue. then it reruns the model and removes the next one with the highest p value. it does this untill it reaches a user defined pvalue. i will use a standard 0.05 for pvalue. i have multiple different models with sligthly different outpout and amount of prints. 


In [12]:
wvs_corr=['A001',
    'A002', 
    'A003', 
    'A004', 
    'A005',
    'A006',
    'A008', 
    'A009', 
    'A170',
    'A173', 
    'C006', 
    'E035',
    'E124',  
    'F028', 
    'F063', 
    'G006',
    'X011',
    'X047_WVS',
    'Y001',
    'Y002',
    'Y003',
    'Y010',
    'Y011',
    'Y011A',
    'Y011B',
    'Y011C',
    'Y012',
    'Y012A',
    'Y012B',
    'Y012C',
    'Y013',
    'Y013A',
    'Y013B',
    'Y013C',
    'Y014',
    'Y014A',
    'Y014B',
    'Y020',
    'Y021',
    'Y021A',
    'Y021B',
    'Y021C',
    'Y022',
    'Y022A',
    'Y022B',
    'Y022C',
    'Y023',
    'Y023A',
    'Y023B',
    'Y023C',
    'Y024',
    'Y024A',
    'Y024B',
    'Y024C',
    'X045_val',
    'Happiness score',
    'oskars_score_with_health',
    'oskars_score']
wvs_corr_measure=['Dailys_per_100000',
    'deppresion_percent',
    'crude_suicide_per_100,000',
    'Life expectancy']
wvs_tot=wvs_corr+wvs_corr_measure

In [13]:
def stepwise_regression(value_column,p_value):
    wvs_corr_temp=wvs_corr.copy()
    wvs_tot_temp=wvs_tot.copy()
    iteration=0
    data_model=data_country[wvs_tot_temp].dropna()
    x = data_model[wvs_corr_temp]
    y = data_model[value_column]
    results = sm.GLS(y, x).fit()
    print(results.summary())
    pvalues=results.pvalues
    while pvalues.max()>p_value:    
        data_pval=pd.DataFrame(pvalues,columns=['pval'])
        print(f"r-squared {round(results.rsquared,3)} for iteration {iteration} with highest pvalue remaning {round(pvalues.max(),3)}, removing {pvalues.idxmax()}")
        wvs_tot_temp.remove(data_pval['pval'].idxmax())
        data_pval=data_pval.drop(data_pval['pval'].idxmax())
        data_model=data_country[wvs_tot_temp].dropna()
        x = data_model[data_pval.index]
        y = data_model[value_column]
        results = sm.GLS(y, x).fit()
        iteration+=1
        pvalues=results.pvalues
    print(results.summary())
    return(data_pval.index)

def stepwise_regression_slim(value_column,p_value):
    wvs_corr_temp=wvs_corr.copy()
    wvs_tot_temp=wvs_tot.copy()
    data_model=data_country[wvs_tot_temp].dropna()
    x = data_model[wvs_corr_temp]
    y = data_model[value_column]
    results = sm.GLS(y, x).fit()
    pvalues=results.pvalues
    while pvalues.max()>p_value:    
        data_pval=pd.DataFrame(pvalues,columns=['pval'])
        wvs_tot_temp.remove(data_pval['pval'].idxmax())
        data_pval=data_pval.drop(data_pval['pval'].idxmax())
        data_model=data_country[wvs_tot_temp].dropna()
        x = data_model[data_pval.index]
        y = data_model[value_column]
        results = sm.GLS(y, x).fit()
        pvalues=results.pvalues
    print(results.summary())
    return(data_pval.index)

def stepwise_regression_slim_list(value_column,p_value):
    wvs_corr_temp=wvs_corr.copy()
    wvs_tot_temp=wvs_tot.copy()
    data_model=data_country[wvs_tot_temp].dropna()
    x = data_model[wvs_corr_temp]
    y = data_model[value_column]
    results = sm.GLS(y, x).fit()
    pvalues=results.pvalues
    while pvalues.max()>p_value:    
        data_pval=pd.DataFrame(pvalues,columns=['pval'])
        wvs_tot_temp.remove(data_pval['pval'].idxmax())
        data_pval=data_pval.drop(data_pval['pval'].idxmax())
        data_model=data_country[wvs_tot_temp].dropna()
        x = data_model[data_pval.index]
        y = data_model[value_column]
        results = sm.GLS(y, x).fit()
        pvalues=results.pvalues
    return(data_pval.index)
    

def stepwise_regression_slim_list(value_column,p_value):
    wvs_corr_temp=wvs_corr.copy()
    wvs_tot_temp=wvs_tot.copy()
    data_model=data_country[wvs_tot_temp].dropna()
    x = data_model[wvs_corr_temp]
    y = data_model[value_column]
    results = sm.GLS(y, x).fit()
    pvalues=results.pvalues
    while pvalues.max()>p_value:    
        data_pval=pd.DataFrame(pvalues,columns=['pval'])
        wvs_tot_temp.remove(data_pval['pval'].idxmax())
        data_pval=data_pval.drop(data_pval['pval'].idxmax())
        data_model=data_country[wvs_tot_temp].dropna()
        x = data_model[data_pval.index]
        y = data_model[value_column]
        results = sm.GLS(y, x).fit()
        pvalues=results.pvalues
    return(data_pval.index)

### def the singe variable linear regresion

this part i define the function where i take a single variable and do a linear analysis. here there is also a mangitud of different variations of the original function with different outpouts. a differance for the original big model is that i have included models with a constant, which does a big differance in the model. i think that all models should have constant, but i didn't get it to play nice with thew big model.

In [14]:
def one_variable_linear_score(variable):
    score=[]
    for values in wvs_corr_measure:
        slim=[]
        slim.append(values)
        slim.append(variable)
        data_model=data_country[slim].dropna()
        x = data_model[[variable]]
        y = data_model[values]
        results = sm.GLS(y, x).fit()
        score.append(results.tvalues[variable])
    return(score)

def one_variable_linear_score_constant(variable):
    score=[]
    for values in wvs_corr_measure:
        slim=[]
        slim.append(values)
        slim.append(variable)
        data_model=data_country[slim].dropna()
        x = data_model[[variable]]
        y = data_model[values]
        x= sm.add_constant(x)
        results = sm.GLS(y, x).fit()
        score.append(results.tvalues[variable])
    return(score)

def one_variable_linear_regression(variable):
    slim=[]
    for values in wvs_corr_measure:
        slim=[]
        slim.append(values)
        slim.append(variable)
        data_model=data_country[slim].dropna()
        x = data_model[[variable]]
        y = data_model[values]
        reg = linear_model.LinearRegression()
        reg.fit(x, y)
        results = sm.GLS(y, x).fit()
        print(results.summary())
        print(reg.score(x, y))

def one_variable_linear_regression_with_constant(variable):
    slim=[]
    for values in wvs_corr_measure:
        slim=[]
        slim.append(values)
        slim.append(variable)
        data_model=data_country[slim].dropna()
        x = data_model[[variable]]
        y = data_model[values]
        x= sm.add_constant(x)
        reg = linear_model.LinearRegression()
        reg.fit(x, y)
        results = sm.GLS(y, x).fit()
        print(results.summary())
        print(reg.score(x, y))

def one_variable_linear_score_comparision(variable):
    score=[]
    for values in wvs_corr_measure:
        slim=[]
        slim.append(values)
        slim.append(variable)
        data_model=data_country[slim].dropna()
        x = data_model[[variable]]
        y = data_model[values]
        reg = linear_model.LinearRegression()
        reg.fit(x, y)
        score.append(reg.score(x, y))
    return(score)

## perfomring analysis with reggresion


first we perfom a stepvis analysis to figure out the best way to construct our linear regression model. first i do a single dependent variable and print out all steps along the way, showing initial model and all the trim down and the final model.    :)

In [ ]:
stepwise_regression('deppresion_percent',0.05)

that is a lot of text and outprint, so binstead of showing every step let's just print out the final model and how good it is to predict the actual score. additionaly i print out the description to the last model. i cycle through the 4 metrics and printy out the model for each one.

In [ ]:
list_r_values=[]
for values in wvs_corr_measure:
    stepwise_regression_slim(values,0.05)
    temp_list=stepwise_regression_slim_list(values,0.05).to_list()
    slim=temp_list+wvs_corr_measure
    data_model=data_country[slim].dropna()
    x = data_model[temp_list]
    y = data_model[values]
    reg = linear_model.LinearRegression()
    reg.fit(x, y)
    print(f"the score value is {round(reg.score(x, y),3)}")
    list_r_values.append(reg.score(x, y))
    print(queastions[['question','description']].loc[queastions['question'].isin(temp_list)])

nice, seams like we have predictrive values from the models. ranging from 0.236-0.818 is a wide span, but are they good? lets compare how well they do compared to single variable. let's create a big table for every single variable and rank how well they predict each dependent variable. let's show it of with an example.

In [ ]:
one_variable_linear_regression('oskars_score_with_health')

Wow, that was truly bad score, let's run thrpough the entierty of the variables and se how each performs.

In [ ]:
df=pd.DataFrame(columns=['variable','Dailys_per_100000',
 'deppresion_percent',
 'crude_suicide_per_100,000',
 'Life expectancy'])

for value in wvs_corr:
    list=[value]
    list.extend(one_variable_linear_score_comparision(value))
    df=pd.concat([df,pd.DataFrame([list], columns=['variable','Dailys_per_100000','deppresion_percent','crude_suicide_per_100,000','Life expectancy'])], ignore_index=True) 

and let's add the regresion model, i saved a list from the original analysis so let's add that on :)

In [19]:
val=['regresion_model']
val.extend(list_r_values)
df=pd.concat([df,pd.DataFrame([val], columns=['variable','Dailys_per_100000','deppresion_percent','crude_suicide_per_100,000','Life expectancy'])], ignore_index=True)

so the best predictor fopr dalys is the regression model, iwth a 0.27 point increase from the second. and we can se that my score is way down the list, so it wasn't a good predictor of dalys.

In [ ]:
df.sort_values(by='Dailys_per_100000',ascending=False)

and for suicide it is a good increase compared to other measures.

In [ ]:
df.sort_values(by='crude_suicide_per_100,000',ascending=False)

so that's where my time and work is done, thank you for following along on this path. hope i have inspired azsome thought. and i must say the happiness rank from World Happiness Report is a very god predictor for most mental health indicator, although it's very bad on one particular thing, suicide.

## extra stuff about my limitations.

first, there a liute note about the correlation matrix not being defined. i think that my propibly is wrong since i haven't provided one. secondly is that i haven't included a constant in my models. below this model is additional information.

In [ ]:
one_variable_linear_regression_with_constant('Happiness score')

as we see addding a constant which is best practice will significantly decreese the effect on the variable. i think i would really need this for the multiple variable analysis but i cant figure out how. where is the t value for each varaible, where a significant correlation is below -2 and above +2.

In [ ]:
df_constant=pd.DataFrame(columns=['variable','Dailys_per_100000',
 'deppresion_percent',
 'crude_suicide_per_100,000',
 'Life expectancy'])

for value in wvs_corr:
    list=[value]
    list.extend(one_variable_linear_score_constant(value))
    df_constant=pd.concat([df_constant,pd.DataFrame([list], columns=['variable','Dailys_per_100000','deppresion_percent','crude_suicide_per_100,000','Life expectancy'])], ignore_index=True)

In [ ]:
df_constant

there is also a lot of extra data that i did not have the time to analys, almost all of the demografic data. i think that i have taken on a bigger bite then i could chew, a bit to optimistic with time and ability 